<a href="https://colab.research.google.com/github/maryoliver28/imers-oalura/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [1]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [2]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [3]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(model=MODEL_ID, contents='Quando é a próxima Imersão IA com Google Gemini da Alura?')

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acompanhar as redes sociais da Alura:** Eles costumam anunciar os eventos nas redes sociais como LinkedIn, Instagram, etc.
*   **Verificar a página de eventos da Alura:** Procure na página da Alura por eventos ou imersões futuras.
*   **Assinar a newsletter da Alura:** Assim você receberá informações sobre os próximos cursos e eventos diretamente no seu e-mail.

In [4]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Response:\n {resposta.text}"))

Response:
 A Imersão IA com Google Gemini da Alura já aconteceu de 12 a 16 de maio de 2025. Fique de olho no site da Alura para futuras edições.

In [5]:
# Exibe a busca
print(f"Busca realizada: {resposta.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in resposta.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(resposta.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima Imersão IA com Google Gemini Alura', 'Alura Imersão IA com Google Gemini datas']
Páginas utilizadas na resposta: eucapacito.com.br, youtube.com



In [6]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [7]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [8]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [10]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name= "agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações no Google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as últimas notícias de lançamentos muito revelantes sobre o tópico abaixo.
      Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiamos das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
      e pode ser substituído por outro que tenha mais.
      Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """

  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [11]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [26]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para um planejador financeiro.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Linkedin sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Linkedin"

    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [27]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Linkedin.
            Por ter um público em geral, use um tom de escrita adequado.
            Revise o rascunho de post de Linkedin abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."

    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [30]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Linkedin com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print ("Você esqueceu de digitar o tópico!")
else:
  print (f"Maravilha! Vamos então criar o post sobre novidades em {topico}")
  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n... Resultado do Agente 1 (Buscador)...\n")
  display(to_markdown(lancamentos_buscados))
  print("........................................................")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n... Resultado do Agente 2 (Planejador)...\n")
  display(to_markdown(plano_de_post))
  print("........................................................")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n...Resultado do Agente 3 (Redator)...\n")
  display(to_markdown(rascunho_de_post))
  print("........................................................")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n... Resultado do Agente 4 (Revisor)...\n")
  display(to_markdown(post_final))
  print("........................................................")



🚀 Iniciando o Sistema de Criação de Posts para Linkedin com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Educação financeira
Maravilha! Vamos então criar o post sobre novidades em Educação financeira

... Resultado do Agente 1 (Buscador)...



> Para identificar os lançamentos mais relevantes em educação financeira no último mês, vou usar o Google para buscar notícias, artigos e anúncios sobre novos cursos, livros, ferramentas ou plataformas que tenham gerado grande interesse ou cobertura na mídia.
> 
> 
> Com base nas minhas buscas, aqui estão alguns dos lançamentos e tendências mais relevantes em educação financeira para maio de 2025:
> 
> 1.  **Eventos e Programas:**
> 
> *   **Semana Nacional de Educação Financeira (ENEF):** Ocorrendo de 12 a 18 de maio de 2025, a ENEF tem como foco a educação financeira para crianças e jovens. Diversas instituições, como o Sicredi e a Capef, estão promovendo ações e iniciativas gratuitas, incluindo cursos, palestras e materiais educativos.
> *   **Bett Brasil 2025:** A DSOP Educação Financeira marcou presença neste evento, que ocorreu de 28 de abril a 1º de maio, com lançamentos como o "Almanaque do Dinheiro" e a coleção didática "Aqui Tem | Matemática com Educação Financeira".
> *   **Programa TOP de Formação de Professores da CVM:** Com inscrições abertas até 30 de maio de 2025, este programa oferece cursos para professores de ensino médio, técnico e superior que lecionam disciplinas relacionadas ao Mercado de Capitais e Educação Financeira.
> *   **Programa Aprender Valor do Banco Central:** Escolas públicas e privadas podem se inscrever neste programa, que oferece recursos educacionais gratuitos e premia as escolas mais engajadas.
> 
> 2.  **Plataformas e Ferramentas Digitais:**
> 
> *   **Plataforma de educação financeira gamificada da Matific:** Lançada na Bett Brasil 2025, esta plataforma é voltada para escolas públicas e privadas e visa transformar o ensino de finanças pessoais por meio de jogos e desafios digitais.
> *   **Curso gamificado de Educação Financeira para Adolescentes do Sicredi:** Lançado durante a Semana ENEF, este curso está disponível na plataforma Sicredi na Comunidade e busca incentivar hábitos financeiros saudáveis entre os jovens.
> *   **Aplicativos de planejamento financeiro:** Há uma crescente oferta de aplicativos que auxiliam no controle de gastos, planejamento de investimentos e organização financeira. O App Plano é um exemplo, disponível gratuitamente nas lojas de aplicativos.
> 
> 3.  **Livros e Publicações:**
> 
> *   **Lançamento do “Almanaque do Dinheiro”:** A DSOP lançou este livro durante a Bett Brasil 2025, com o objetivo de educar as novas gerações sobre finanças pessoais de maneira lúdica e acessível.
> *   **Listas de livros recomendados sobre educação financeira:** Diversos artigos e publicações listam livros essenciais para quem busca aprender sobre finanças pessoais, investimentos e planejamento financeiro. Alguns títulos populares incluem "Pai Rico, Pai Pobre" de Robert Kiyosaki, "Me Poupe!" de Nathalia Arcuri e "O Investidor Inteligente" de Benjamin Graham.
> 
> 4.  **Tendências e Abordagens:**
> 
> *   **Gamificação:** O uso de jogos e desafios digitais para tornar o aprendizado sobre finanças mais interessante e envolvente, especialmente para crianças e jovens.
> *   **Personalização do aprendizado:** A criação de experiências educativas adaptadas às necessidades individuais de cada usuário, por meio de ferramentas de inteligência artificial.
> *   **Consumo consciente:** A promoção de escolhas financeiras mais responsáveis e alinhadas com os valores e prioridades de cada pessoa.
> *   **Digitalização:** O uso de ferramentas e tecnologias digitais para facilitar o planejamento, acompanhamento e gestão das finanças pessoais.
> 
> Esses são apenas alguns dos lançamentos e tendências mais relevantes em educação financeira em maio de 2025. A área está em constante evolução, com novas ferramentas, plataformas e abordagens surgindo para ajudar as pessoas a tomar decisões financeiras mais conscientes e responsáveis.


........................................................

... Resultado do Agente 2 (Planejador)...



> Com base nos lançamentos e tendências em educação financeira para maio de 2025, podemos criar um plano de conteúdo para redes sociais abordando os temas mais relevantes.
> 
> **Google Searches:**
> 
> 
> Com base nos resultados da pesquisa, o tema mais relevante para um post de rede social sobre educação financeira em maio de 2025 é a **Semana Nacional de Educação Financeira (ENEF)**, que ocorreu de 12 a 18 de maio, com foco em crianças e jovens.
> 
> **Pontos Relevantes:**
> 
> *   **Foco em crianças e jovens:** A ENEF 2025 teve como tema central a educação financeira para crianças e jovens, preparando-os para escolhas conscientes.
> *   **Iniciativas gratuitas:** Diversas instituições ofereceram cursos, palestras e materiais educativos gratuitos durante a semana.
> *   **Engajamento escolar:** Escolas públicas e privadas foram incentivadas a participar, com programas e recursos educacionais disponíveis.
> *   **Hábitos financeiros saudáveis:** O objetivo principal foi promover hábitos financeiros saudáveis e escolhas responsáveis entre os jovens.
> *   **Conscientização:** A conscientização sobre a importância do planejamento financeiro para desenvolver uma relação equilibrada com o dinheiro.
> 
> **Plano de Conteúdo:**
> 
> 1.  **Introdução:**
>     *   Começar o post com uma pergunta que chame a atenção do público, como: "Você sabia que a educação financeira pode começar desde a infância?".
>     *   Apresentar a Semana Nacional de Educação Financeira (ENEF) como um evento importante para promover a educação financeira no Brasil.
>     *   Destacar o tema da ENEF 2025: "Educação Financeira para Crianças e Jovens: Preparando a Sociedade para Escolhas Conscientes".
> 
> 2.  **Importância da Educação Financeira para Crianças e Jovens:**
>     *   Explicar por que é fundamental ensinar educação financeira desde cedo.
>     *   Mencionar como a educação financeira pode ajudar crianças e jovens a desenvolverem hábitos financeiros saudáveis, como poupar, planejar e consumir de forma consciente.
>     *   Citar dados estatísticos sobre a falta de conhecimento financeiro entre jovens e adultos no Brasil.
> 
> 3.  **Destaques da ENEF 2025:**
>     *   Apresentar algumas das principais iniciativas e programas lançados durante a ENEF 2025, como:
>         *   Cursos gamificados de educação financeira para adolescentes (Sicredi).
>         *   Plataformas de educação financeira gamificadas para escolas (Matific).
>         *   Lançamento do "Almanaque do Dinheiro" (DSOP).
>         *   Programa Aprender Valor do Banco Central.
>         *   Programa TOP de Formação de Professores da CVM.
>     *   Incluir links para que o público possa acessar esses recursos.
> 
> 4.  **Tendências em Educação Financeira:**
>     *   Abordar as principais tendências em educação financeira, como:
>         *   Gamificação: O uso de jogos e desafios digitais para tornar o aprendizado mais interessante e envolvente.
>         *   Personalização do aprendizado: A criação de experiências educativas adaptadas às necessidades individuais de cada usuário.
>         *   Consumo consciente: A promoção de escolhas financeiras mais responsáveis e alinhadas com os valores de cada pessoa.
>         *   Digitalização: O uso de ferramentas e tecnologias digitais para facilitar o planejamento, acompanhamento e gestão das finanças pessoais.
> 
> 5.  **Dicas Práticas para Pais e Educadores:**
>     *   Oferecer dicas práticas para pais e educadores sobre como ensinar educação financeira para crianças e jovens, como:
>         *   Envolver as crianças no planejamento financeiro familiar.
>         *   Mostrar aos filhos de onde vem o dinheiro e como ele é usado no dia a dia.
>         *   Incentivar a poupança desde cedo.
>         *   Discutir sobre consumo consciente e a diferença entre necessidades e desejos.
>         *   Utilizar jogos e atividades lúdicas para ensinar conceitos financeiros.
> 
> 6.  **Conclusão:**
>     *   Reforçar a importância da educação financeira para construir um futuro mais próspero e sustentável.
>     *   Incentivar o público a buscar mais informações e recursos sobre educação financeira.
>     *   Finalizar com uma frase inspiradora sobre o poder da educação financeira para transformar vidas.
> 
> **Hashtags:**
> #EducaçãoFinanceira #ENEF2025 #FinançasParaCrianças #FinançasParaJovens #PlanejamentoFinanceiro #ConsumoConsciente #Gamificação #Digitalização #FuturoFinanceiro
> 
> Este plano de conteúdo visa fornecer informações relevantes e úteis sobre educação financeira, incentivando o público a tomar decisões mais conscientes e responsáveis em relação ao dinheiro.
> 


........................................................

...Resultado do Agente 3 (Redator)...



> Você sabia que a educação financeira pode começar desde a infância? 💰👧👦
> 
> A Semana Nacional de Educação Financeira (ENEF) 2025 já passou, mas o aprendizado continua! Este ano, o tema foi "Educação Financeira para Crianças e Jovens: Preparando a Sociedade para Escolhas Conscientes". E por que isso é tão importante? 🤔
> 
> Ensinar educação financeira desde cedo ajuda a garotada a desenvolver hábitos financeiros saudáveis, como poupar, planejar e consumir de forma consciente. E os benefícios são enormes:
> 
> ✅ Maior controle sobre o próprio dinheiro.
> ✅ Capacidade de tomar decisões financeiras mais inteligentes.
> ✅ Preparação para um futuro financeiro mais seguro e estável.
> 
> Durante a ENEF 2025, tivemos diversas iniciativas incríveis, como cursos gamificados, plataformas educativas para escolas e o lançamento de materiais didáticos. A gamificação, a personalização do aprendizado e o consumo consciente são tendências que vieram para ficar! 🚀
> 
> E você, como tem ensinado educação financeira para as crianças e jovens ao seu redor? Compartilhe suas dicas nos comentários! 👇
> 
> #EducaçãoFinanceira #ENEF2025 #FinançasParaCrianças #PlanejamentoFinanceiro
> 


........................................................

... Resultado do Agente 4 (Revisor)...



> O rascunho está ótimo, mas podemos fazer alguns ajustes para deixá-lo ainda mais claro e envolvente:
> 
> **Problemas e Sugestões:**
> 
> 1.  **Semana Nacional de Educação Financeira (ENEF) 2025 já passou:** Como a ENEF de 2025 ainda não aconteceu, sugiro remover a menção de que ela já passou. Podemos mudar para uma abordagem mais geral sobre a importância da educação financeira desde a infância, sem amarrar a um evento futuro.
> 2.  **Emojis:** O uso de emojis está bom, mas podemos adicionar mais um ou dois para dar mais vida ao post.
> 3.  **Chamada para Ação:** A chamada para ação no final ("E você, como tem ensinado educação financeira para as crianças e jovens ao seu redor? Compartilhe suas dicas nos comentários!") é boa, mas podemos torná-la mais específica e convidativa.
> 
> **Sugestão de revisão:**
> 
> Você sabia que a educação financeira pode começar desde a infância? 💰👧👦
> 
> Ensinar educação financeira desde cedo ajuda a garotada a desenvolver hábitos financeiros saudáveis, como poupar, planejar e consumir de forma consciente. E por que isso é tão importante? 🤔
> 
> Os benefícios são enormes:
> 
> ✅ Maior controle sobre o próprio dinheiro.
> ✅ Capacidade de tomar decisões financeiras mais inteligentes.
> ✅ Preparação para um futuro financeiro mais seguro e estável.
> 
> A gamificação, a personalização do aprendizado e o consumo consciente são tendências que vieram para ficar! 🚀
> 
> Que tal começar hoje mesmo? Compartilhe nos comentários uma dica simples de como você ensinaria ou aprendeu sobre educação financeira! 👇 😉
> 
> #EducaçãoFinanceira #FinançasParaCrianças #PlanejamentoFinanceiro #FuturoFinanceiro
> 


........................................................
